In [18]:
import pandas as pd

all_tweets_df = pd.read_csv('reddit_set.csv')
# all_tweets_df = all_tweets_df[:1000]

In [19]:
all_tweets_df

,Unnamed: 0,PID,Text_data,Label
0,0,dev_pid_1,"I enjoyed today, and I still am! Tomorrows dep...",1
1,1,dev_pid_2,I sorta tried to kill myself : I had a total b...,1
2,2,dev_pid_3,Best suicide method? : I like it quick and eas...,1
3,3,dev_pid_4,a story : I remember the time I'd get on my 3D...,1
4,4,dev_pid_5,The world only cares about beautiful people : ...,1
...,...,...,...,...
5867,8886,train_pid_8887,Ways to reverse memory loss from depression? :...,2
5868,8887,train_pid_8888,A Comprehensive Guide To Slowly Getting Better...,2
5869,8888,train_pid_8889,I don’t think college is right for me : TW: su...,2
5870,8889,train_pid_8890,Please help: Severe insomnia affecting me in m...,2


In [20]:
from sklearn.model_selection import train_test_split

# Training = 0.7, validation = 0.1, test = 0.2
training_tweets, temp_tweets, training_labels, temp_labels = train_test_split(list(all_tweets_df['Text_data']),list(all_tweets_df['Label']), test_size=0.3, random_state=123)
val_tweets, test_tweets, val_labels, test_labels = train_test_split(temp_tweets, temp_labels, test_size=(1/3), random_state=123)

In [11]:
from transformers import ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
# tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
def tokenize_function(dataset):
    model_inputs = tokenizer(dataset['text'], padding="max_length", truncation=True, max_length=1000)
    return model_inputs


loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/vocab.txt from cache at /Users/brookegrantham/.cache/huggingface/transformers/ece45ade3e01224cf31fed8e183b306d17b84e8abd415363474cfe72274f7814.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/tokenizer_config.json from cache at /Users/brookegrantham/.cache/huggingface/transformers/8b3aea9f7242b3d19268df5b1bfed8f66e08671a72ac0809ada08e5ef1adc592.19eda9a6da5fb0e52a45200c95876729561dde16a69b9116953af6edca1d1e92
loading configuration file https://huggingface.co/google/electra-small-discriminator/resolve/main/config.json from cache at /Users/brookegrantham/.cache/hug

In [12]:
train = {'text': training_tweets, 'label': training_labels}
val = {'text': val_tweets, 'label': val_labels}
test = {'text': test_tweets, 'label': test_labels}

In [13]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train)
val_dataset = Dataset.from_dict(val)
test_dataset = Dataset.from_dict(test)

Issue: There was NaN in the dataset not allowing it to be transfered to a Dataset type, thus checking the data condition in rough cells below.

In [14]:
# integers = [elm for elm in all_tweets_df['condition'] if not (isinstance(elm, int))]
# print(len(integers))

In [15]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="transformer_checkpoints",
    num_train_epochs=1,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
from transformers import Trainer, ElectraForSequenceClassification

model = ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator', num_labels=3)

for param in model.electra.parameters():
    param.requires_grad = False


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

loading configuration file https://huggingface.co/google/electra-small-discriminator/resolve/main/config.json from cache at /Users/brookegrantham/.cache/huggingface/transformers/ca13c16218c6780ec76753d3afa19fcb7cc759e3f63ee87e441562d374762b3d.3dd1921e571dfa18c0bdaa17b9b38f111097812281989b1cb22263738e66ef73
Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation"

RuntimeError: The size of tensor a (1000) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
import numpy as np
import torch

def predict_nn(trained_model, test_dataset):

    output = trained_model(attention_mask=torch.tensor(test_dataset["attention_mask"]), input_ids=torch.tensor(test_dataset["input_ids"]))

    pred_labs = np.argmax(output["logits"].detach().numpy(), axis=1)

    gold_labs = test_dataset["label"]

    return gold_labs, pred_labs

gold_labs, pred_labs = predict_nn(model, test_dataset)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
print("ELECTRA on the John Hopkins Twitter dataset:")
print(f'The accuracy score is {accuracy_score(pred_labs, gold_labs)}')
print(f'The f1-score is {f1_score(pred_labs,gold_labs)}')

In [ ]:
pd.DataFrame(({'pred':pred_labs, 'gold':gold_labs, 'text':test_tweets}))